In [8]:
from google.colab import files
uploaded = files.upload()

Saving asmm_engg.csv to asmm_engg (2).csv


In [9]:
filename = list(uploaded.keys())[0]
print("Uploaded file:", filename)

Uploaded file: asmm_engg (2).csv


In [10]:
import pandas as pd

df = pd.read_csv(filename)

print(df.head())
print("Rows:", len(df))

                                                 asm  \
0  গ্ৰেটাৰ নইডা ৱেষ্টৰ অসুবিধাসমূহ কমাৰ নামেই নোল...   
1  নিজৰ দাবীসমূহক লৈ নিৰ্মাণ কাৰ্য্য বন্ধ কৰাবৰ ক...   
2  দেওবাৰে মাইচা গাঁৱত কৃষক সংঘৰ্ষ সমিতিয়ে পঞ্চা...   
3  ক্ষতিপূৰণ দিয়াৰ নামত প্ৰাধিকৰণ আৰু এডিএম এলএ ...   
4  পঞ্চায়তত কৃষকসকলে কলে যে প্ৰাধিকৰণ কৃষকসকলৰ স...   

                                                 eng  
0  the problems  of greater noida west  are  not ...  
1  there  is  rush  among the farmer organization...  
2  the farmer struggle committee  after calling  ...  
3  in the name  of giving  compensation  ,  the a...  
4  the farmers  have told  the panchayat  that  t...  
Rows: 70000


In [12]:
print(df.columns)

Index(['asm', 'eng'], dtype='object')


In [13]:
sample_text = ""

for line in df["asm"].astype(str).head(2000):
    sample_text += line + "\n"

print("Sample length:", len(sample_text))


Sample length: 156170


In [16]:
import pandas as pd

def load_small_sample():
    sample_text = ""

    # read CSV safely
    df = pd.read_csv(filename, encoding="utf-8")

    # 🔹 CHANGE 'text' to your actual column name if different
    text_column = df.columns[0]  # auto-picks first column

    for j, line in enumerate(df[text_column].astype(str)):
        sample_text += line + "\n"
        if j > 2000:
            break

    return sample_text


# Step 3 — Run
raw_text = load_small_sample()
print("Sample length:", len(raw_text))

Sample length: 156306


In [18]:
!pip install tiktoken

import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

encoded = tokenizer.encode(raw_text)
print("Encoded tokens:", len(encoded))

Encoded tokens: 318973


In [19]:

import torch
from torch.utils.data import Dataset, DataLoader

class GPTDataset(Dataset):
    def __init__(self, token_ids, max_length=64, stride=32):
        self.input_ids = []
        self.target_ids = []

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1:i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

dataset = GPTDataset(encoded, max_length=64, stride=32)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

batch = next(iter(dataloader))
print("Input batch shape:", batch[0].shape)
print("Target batch shape:", batch[1].shape)

Input batch shape: torch.Size([8, 64])
Target batch shape: torch.Size([8, 64])


In [20]:

vocab_size = tokenizer.n_vocab
embedding_dim = 128

embedding = torch.nn.Embedding(vocab_size, embedding_dim)
sample_input = batch[0]
embedded_output = embedding(sample_input)

print("Embedded output shape:", embedded_output.shape)

Embedded output shape: torch.Size([8, 64, 128])


In [21]:

batch = next(iter(dataloader))
inputs, targets = batch

print("INPUT TEXT:")
print(tokenizer.decode(inputs[0].tolist()))

print("\nTARGET TEXT:")
print(tokenizer.decode(targets[0].tolist()))

INPUT TEXT:
�� উৎপাদন কৰা হয় ।
মচুৰৰ খেতি মু�

TARGET TEXT:
� উৎপাদন কৰা হয় ।
মচুৰৰ খেতি মুখ


In [22]:

input_ids = inputs[0]
target_ids = targets[0]

for i in range(5):
    print(
        tokenizer.decode([input_ids[i].item()]),
        "→",
        tokenizer.decode([target_ids[i].item()])
    )

� → �
� →  
  → �
� → �
� → �


In [23]:
for i in range(5):
    print(input_ids[i].item(), "→", target_ids[i].item())

100 → 229
229 → 220
220 → 48071
48071 → 231
231 → 156


In [24]:
for i in range(10, 16):
    print(
        tokenizer.decode(input_ids[:i].tolist()),
        "→",
        tokenizer.decode(target_ids[:i].tolist())
    )

�� উৎপ → � উৎপ�
�� উৎপ� → � উৎপা
�� উৎপা → � উৎপা�
�� উৎপা� → � উৎপাদ
�� উৎপাদ → � উৎপাদ�
�� উৎপাদ� → � উৎপাদন


In [25]:
print("Embedding weight shape:", embedding.weight.shape)
print("Sample input shape:", inputs.shape)
print("Embedded output shape:", embedded_output.shape)

Embedding weight shape: torch.Size([50257, 128])
Sample input shape: torch.Size([8, 64])
Embedded output shape: torch.Size([8, 64, 128])


In [26]:
lm_head = torch.nn.Linear(embedding_dim, vocab_size)

logits = lm_head(embedded_output)

print("Logits shape:", logits.shape)

Logits shape: torch.Size([8, 64, 50257])
